In [ ]:
import os
import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import torchvision
import torch.nn as nn
from torch import optim
from sklearn.metrics import classification_report
import torchvision.io as io
import pydicom
import cv2
import numpy as np
import dicom2jpg

In [ ]:
os.getcwd()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def get_data(path):
    
    images = []
    targets = []
    for idx , x in enumerate(path):
            image_list = os.listdir(x)
            images += [x + each_image for each_image in image_list]
            targets += [0 for _ in image_list]
    return images , targets

In [ ]:
path = [os.getcwd() + '/Practice_DICOM/chestimages_DICOM/', os.getcwd() +'/Practice_PNGandJPG/chestimage_JPG/' , os.getcwd() + '/Practice_PNGandJPG/chestimage_PNG/' ]

In [ ]:
train_images , train_targets = get_data(path)

In [ ]:
transform =  transforms.Compose([transforms.ToTensor()])

In [ ]:
def read_image_file(path)-> torch.Tensor:
        image_locations_tensor = []
        
        for index , rows in path.iterrows():

            print(rows.values[0].split(".")[0])
            
            if rows[0].split(".")[1] in ["jpeg" , "png"]:
                img = Image.open(rows.values[0])
                image_locations_tensor += [transform(img)]
            elif rows[0].split(".")[1] in ["dcm"]:
                dcm_image = pydicom.dcmread(rows[0])
                image = dcm_image.pixel_array.astype(float)
                rescaled_image = (np.maximum(image,0)/image.max())*255 
                final_image = np.uint8(rescaled_image)
                final_image = Image.fromarray(final_image) 
               
                final_image.save(rows.values[0].split(".")[0] +'.jpg')
                img = Image.open(rows.values[0].split(".")[0] +'.jpg')
                image_locations_tensor += [transform(img)]
            
        return torch.vstack(image_locations_tensor)
        
def read_label_file(target)-> torch.Tensor:
        labels = target[0]
        labels_tensor = torch.tensor(labels)
        return labels_tensor

In [ ]:
class InputData(Dataset):
    def __init__(self, path , target , transform  = None):
        self.path  = path
        self.targets_columns = target
        self.transform = transform
        self.data , self.targets = self._load_data(self.path , self.targets_columns)

    
    def _load_data(self , path , target):
        image_file = path
        data = read_image_file(path)

        label_file = target
        targets = read_label_file(target)
        
        return data , targets
        
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.path.iloc[idx,0]
        img = Image.open(image)
        
        if self.transform:
            data = self.transform(img)
        
        target = int(self.targets_columns.iloc[idx,0])
            
        return (data , target)

In [ ]:
! pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm
! pip install --upgrade pydicom

In [ ]:
! pip install dicom2jpg

In [ ]:
train_data  = InputData(pd.DataFrame(train_images) , pd.DataFrame(train_targets) , transform)

In [ ]:
train_data.data[0]

In [ ]:
train_data.targets.shape

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_data , batch_size = 2 , shuffle = True , num_workers = 4)

In [ ]:

plt.imshow(train_data.data[9], cmap='gray')
plt.title(f"{train_data.targets[9]}")
plt.axis('off')
plt.show()